# Decorator Playground

~~果然还是得上notebook~~

装饰器是一种用于修改函数或类行为的**设计模式**。它们通常以函数或类的形式出现。

**警告**：此部分内容全是个人理解，请以[官方文档](https://docs.python.org/zh-cn/3/glossary.html#term-decorator)为准

核心机制：装饰器接受一个函数或类作为输入，并返回一个经过修改或增强的对象来**替换原来位置的定义**，以便在不改变原始代码的情况下添加额外的行为。多个装饰器会以嵌套方式被应用。

## 注意

尽管在类或函数定义的上一行使用`@`符紧接装饰器（e.g. `@wrapper`）能方便地装饰这个类或函数，人们也都是这样做的，但 ——

`@`符只是一个语法糖。下面的两个单元格中的代码在语义上完全等价。

—— 希望这有助于你的理解。

In [ ]:
def f(arg):
    ...
f = wrapper(f)

In [ ]:
@wrapper
def f(arg):
    ...

（嗯这里的`wrapper`指的是一个装饰器）

## 装饰器函数

此种装饰器以函数的形式存在。它接收一个函数，返回一个被修改/新的函数，或者接收一个类，返回一个被修改的类。

### 装饰函数的装饰器函数

最常用。用于装饰一个函数的函数。一般用于在函数执行前后添加行为（比如记日志、做资源检查与释放、重试什么的）

一个简单的例子如下，实现了一个**最简单的**重试装饰器：

In [4]:
def retry(func):
    def wrapped(*args, **kwargs):
        while True:
            try:
                return func(*args, **kwargs)  # <- 原始函数在这里！
            except Exception as e:
                pass
    return wrapped

> 试着找出这个装饰器的问题（答案藏在本节某处喵w）
> 
> 再将它与第三方库[`retry`](https://pypi.org/project/retry/)中成熟的重试装饰器做对比，看看大佬是怎么做的。（如果愿意的话还可以去翻翻它的 ~~底裤~~ 源代码，看看具体是怎么实现的）

我们来试试使用这个简单的装饰器吧。

这是一个`trouble_maker`，它有 `1/3` 的概率报错。

尝试在不同的函数定义下（使用装饰器和不使用装饰器）反复运行它们，看看会不会报错、

In [5]:
def trouble_maker(msg):
    from random import randint
    if randint(1, 3) == 1:
        raise RuntimeError(msg)

In [ ]:
@retry
def trouble_maker(msg):
    from random import randint
    if randint(1, 3) == 1:
        raise RuntimeError(msg)

In [12]:
trouble_maker("wwwwwwwwww")

~~好一个 trouble maker，笑死~~

> 包括装饰函数的装饰器函数在内的，返回值为函数或参数含有函数的函数，被称作高阶函数。Python中有一个专门的标准库对这样的函数提供支持：[functools](https://docs.python.org/zh-cn/3/library/functools.html)

### 装饰类的装饰器函数

用于装饰一个类的函数。与上面那哥们不同，它一般用于给类添加方法或属性。

一个简单的例子如下：

In [1]:
def add_method(cls):
    def new_method(self):
        print("new method of class", cls, ", instance", self)
        print("name:", self.name)
    cls.new_method = new_method
    return cls

它给传入的类添加了一个新方法，并假设传入的类拥有`name`这个属性。调用此方法时会直接打印所属类、所属实例和`name`属性。

让我们试着使用它。

In [2]:
@add_method
class Kelas:
    def __init__(self, name):
        self.name = name

In [3]:
k = Kelas("Nyan")
k.new_method()

new method of class <class '__main__.Kelas'> , instance <__main__.Kelas object at 0x000001E106C94350>
name: Nyan


## 装饰器类

此种装饰器以类的形式出现。与装饰器函数的机制差不多，但写法不同，自由度也更高。（也用得相对很少）

前面提到，调用装饰器返回的东西会用来替换原来位置的定义。装饰器类也一样，它使用自己实例化产生的实例替代被装饰的类或函数，原始类/函数被保存为实例的一个属性。

### 装饰函数的装饰器类

装饰器类实例化产生的实例必须带有`__call__`方法，这个特殊方法能让实例变得可被调用，不然也就无法替代原始函数了。

一个简单的例子如下。这个例子展示了一个用于计数调用次数的装饰器类。使用方法仍然与先前的一致。

将`__call__`方法注释掉再试试，看看会发生什么。

In [13]:
class CountCalls:
    def __init__(self, func):
        self.func = func
        self.call_count = 0

    def __call__(self, *args, **kwargs):
        self.call_count += 1
        print(f"Calling {self.func.__name__} {self.call_count} times")
        return self.func(*args, **kwargs)

In [14]:
@CountCalls
def say_hi():
    print("Hi!")

In [38]:
say_hi()

Calling say_hi 24 times
Hi!


### 装饰类的装饰器类

装饰器类产生的实例会替代原始类，这个实例的`__call__`方法会代理原始类的`__init__`方法。

同样是一个简单的例子。这个例子展示了一个用于计数实例化次数的装饰器类。使用方法仍然与先前的一致。

In [39]:
class CountInstance:
    def __init__(self, cls):
        self.cls = cls
        self.inst_count = 0

    def __call__(self, *args, **kwargs):
        self.inst_count += 1
        print(f"Instantiating {self.cls.__name__} {self.inst_count} times")
        return self.cls(*args, **kwargs)

In [40]:
@CountInstance
class Cat:
    def __init__(self) -> None:
        print("A new kitten is born!")

In [45]:
if "cats" in vars():
    cats.append(Cat())
else:
    cats = [Cat()]

Instantiating Cat 5 times
A new kitten is born!


## 一些内置装饰器

> 虽然官方文档里「装饰的概念也适用于类，但通常较少这样使用」这样说了，但是标准库里可还是有一堆用类实现的装饰器啊ww

（仅列举出部分）

| 装饰器 | 说明 |
| ----- | ---- |
| `staticmethod` | 将一个方法声明为[静态方法](https://docs.python.org/zh-cn/3/library/functions.html#staticmethod) |
| `classmethod` | 将一个方法声明为[类方法](https://docs.python.org/zh-cn/3/library/functions.html#classmethod) |
| `property` | 将一个方法装饰为[`property`属性](https://docs.python.org/zh-cn/3/library/functions.html#property) |
| `abc.abstractmethod` | 将一个方法声明为[抽象方法](https://docs.python.org/zh-cn/3/library/abc.html#abc.abstractmethod) |
| `functools.wraps` | 用于在装饰器定义中复制原函数的元数据，参见[官方文档](https://docs.python.org/zh-cn/3/library/functools.html#functools.wraps) |
| `functools.singledispatch` | 用于方便地定义泛型函数，参见[官方文档](https://docs.python.org/zh-cn/3/library/functools.html#functools.singledispatch) |